## recommendation system with knn
- cols = beer_name, review_profilename, review_overall, review_aroma, review_appearance

In [44]:
import pandas as pd

path = '/Users/gwag-eunji/Downloads/beer_reviews.csv'
data = pd.read_csv(path)
data.head()
len(data.columns)

13

In [46]:
re_data = data.iloc[1:10001]
re_data.tail()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
9996,21590,Ram Restaurant & Brewery,1259451373,3.5,3.5,3.5,jdense,American IPA,3.5,3.0,Big Red IPA,6.0,54411
9997,5049,Bath Ales Ltd,1277680856,3.5,3.5,3.5,AEK,English Bitter,3.5,3.5,Golden Hare,4.4,52239
9998,5049,Bath Ales Ltd,1205875855,4.0,4.0,4.0,wl0307,English Bitter,3.5,4.0,SPA,3.7,41647
9999,5049,Bath Ales Ltd,1312801866,3.5,4.0,4.5,fullsweep,English Bitter,3.5,3.5,Wild Hare,5.0,34416
10000,5049,Bath Ales Ltd,1238938746,4.0,4.0,3.0,BlackHaddock,English Bitter,4.0,4.5,Wild Hare,5.0,34416


In [60]:
user_info = re_data.iloc[:,-7:-6]
beer_info = re_data.iloc[:, -3:-2]
rating_info = re_data.iloc[:, 3:6]
user_info
rating_info

                    beer_name
1                    Red Moon
2      Black Horse Black Beer
3                  Sausa Pils
4               Cauldron DIPA
5         Caldera Ginger Beer
...                       ...
9996              Big Red IPA
9997              Golden Hare
9998                      SPA
9999                Wild Hare
10000               Wild Hare

[10000 rows x 1 columns]


,review_overall,review_aroma,review_appearance
1,3.0,2.5,3.0
2,3.0,2.5,3.0
3,3.0,3.0,3.5
4,4.0,4.5,4.0
5,3.0,3.5,3.5
...,...,...,...
9996,3.5,3.5,3.5
9997,3.5,3.5,3.5
9998,4.0,4.0,4.0
9999,3.5,4.0,4.5


In [61]:
concat_data = pd.concat([user_info, beer_info, rating_info], axis=1)
concat_data.head()

,review_profilename,beer_name,review_overall,review_aroma,review_appearance
1,stcules,Red Moon,3.0,2.5,3.0
2,stcules,Black Horse Black Beer,3.0,2.5,3.0
3,stcules,Sausa Pils,3.0,3.0,3.5
4,johnmichaelsen,Cauldron DIPA,4.0,4.5,4.0
5,oline73,Caldera Ginger Beer,3.0,3.5,3.5


In [69]:
user_rating = concat_data.drop_duplicates(['beer_name', 'review_profilename'])
user_rating.head()
user_rating_pivot = user_rating.pivot(index='beer_name', columns='review_profilename', values='review_overall').fillna(0)
user_rating_pivot.shape

(945, 3192)

In [71]:
from scipy.sparse import csr_matrix
user_rating_matrix = csr_matrix(user_rating_pivot.values)
user_rating_matrix
user_rating_pivot.shape

(945, 3192)

In [73]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [74]:
import numpy as np
query_index = np.random.choice(user_rating_pivot.shape[0])
print(user_rating_pivot.iloc[query_index,:])

review_profilename
NaN         0.0
05Harley    0.0
1759Girl    0.0
1Adam12     0.0
1fastz28    0.0
           ... 
zimm421     0.0
zoolzoo     0.0
zoso1967    0.0
zrab11      0.0
zuggy9      0.0
Name: Batch 1300, Length: 3192, dtype: float64


In [81]:
distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[query_index,:].values.reshape(1, -1)
                                         , n_neighbors=6)
print('Distances=',distances, 'Indices=', indices)
print(distances.flatten())
print(len(distances.flatten()))

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('recommendation for {0}:\n'.format(user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distacne of {2}:'.format(i, user_rating_pivot.index[indices.flatten()[i]],
                                                      distances.flatten()[i]) )

Distances= [[0.         0.21064778 0.29289322 0.34149539 0.60683317 0.70144265]] Indices= [[ 84 369 776 332 753 751]]
[0.         0.21064778 0.29289322 0.34149539 0.60683317 0.70144265]
6
recommendation for Batch 1300:

1: Halliburton On Trent, with distacne of 0.21064778262367367:
2: Smoke Of A Beech Smoked IPA, with distacne of 0.29289321881345254:
3: Frambozen, with distacne of 0.34149539213148195:
4: Scotch Ale, with distacne of 0.6068331745025329:
5: Schwarz Beer, with distacne of 0.7014426516551696:


In [88]:
user_rating.head()
user_rating.shape

(9932, 5)

In [94]:
train_dataset = user_rating.iloc[0:6001]
val_dataset = user_rating.iloc[6001: 7000]
test_dataset = user_rating.iloc[7000:]
test_dataset.head()

,review_profilename,beer_name,review_overall,review_aroma,review_appearance
7022,Absumaster,La Binchoise Blonde Tradition,4.0,3.5,4.0
7023,jwc215,La Binchoise Blonde Tradition,4.5,4.0,4.5
7024,djeucalyptus,La Binchoise Blonde Tradition,4.5,4.0,4.5
7025,gregmurer,La Binchoise Blonde Tradition,5.0,4.0,4.5
7026,Nipitato,La Binchoise Blonde Tradition,4.5,5.0,3.5
